In [1]:
import gzip
import pickle 
import mplhep as hep
hep.style.use("CMS")
import matplotlib
matplotlib.rc('font', size=15)
import numpy as np
import torch 
import pandas as pd
import numpy as np
import plotly.express as px
import sys
sys.path.append("/afs/cern.ch/work/m/mgarciam/private/mlpf/")
from sklearn.cluster import DBSCAN, HDBSCAN
from torch_scatter import scatter_max, scatter_add, scatter_mean
from src.layers.inference_oc import get_clustering, obtain_intersection_values
from scipy.optimize import linear_sum_assignment
import numpy as np
import matplotlib.pyplot as plt

/afs/cern.ch/work/m/mgarciam/private/miniconda/miniconda3/envs/graphgps0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.utils.inference.pandas_helpers import open_mlpf_dataframe
list_all_df_hdb = []
path_hgcal = "/eos/user/m/mgarciam/datasets_mlpf/models_trained_CLD/eval_comp_05/showers_df_evaluation/0_0_None_hdbscan_option9_v1.pt"
sd_hgb, matched_hgb = open_mlpf_dataframe(path_hgcal, False)
path_hgcal = "/eos/user/m/mgarciam/datasets_mlpf/models_trained_CLD/eval_comp_05/showers_df_evaluation/0_0_None_pandora_option9_v1.pt"
sd_pandora, matched_hgbq1 = open_mlpf_dataframe(path_hgcal, False)

In [6]:
sd_pandora.keys()

Index(['true_showers_E', 'reco_showers_E', 'pred_showers_E',
       'e_pred_and_truth', 'pandora_calibrated_E', 'pandora_calibrated_pfo',
       'pandora_calibrated_pos', 'pandora_ref_pt', 'pid', 'pandora_pid',
       'step', 'number_batch', 'is_track_in_cluster', 'is_track_correct',
       'is_track_in_MC', 'distance_to_cluster_MC', 'vertex', 'ECAL_hits',
       'HCAL_hits', 'true_pos', 'pid_4_class_true'],
      dtype='object')

In [7]:
sd_pandora

,true_showers_E,reco_showers_E,pred_showers_E,e_pred_and_truth,pandora_calibrated_E,pandora_calibrated_pfo,pandora_calibrated_pos,pandora_ref_pt,pid,pandora_pid,...,number_batch,is_track_in_cluster,is_track_correct,is_track_in_MC,distance_to_cluster_MC,vertex,ECAL_hits,HCAL_hits,true_pos,pid_4_class_true
0,3.094387,2.009781,1.954067,1.954067,3.867040,3.116272,"[0.06580160558223724, -2.7740938663482666, -1....","[0.3283114433288574, -13.26000690460205, -6.75...",13.0,13.0,...,0.0,1.0,1.0,1,0.114658,"[0.0, 0.0, 0.0]",31.0,45.0,"[0.06361676007509232, -2.7549726963043213, -1....",4.0
1,7.807354,8.110796,7.910188,7.893300,8.019310,7.757472,"[1.4460123777389526, -7.053318500518799, -2.88...","[2.700740337371826, -13.260008811950684, -5.41...",-11.0,-11.0,...,0.0,1.0,1.0,1,0.092675,"[0.0, 0.0, 0.0]",255.0,0.0,"[1.454261064529419, -7.099344730377197, -2.905...",0.0
2,3.398957,1.745717,1.622844,1.622844,3.414821,3.394703,"[1.977081060409546, -2.729095697402954, -0.384...","[8.009926795959473, -11.034708976745605, -1.55...",13.0,-211.0,...,0.0,1.0,1.0,1,0.101245,"[0.0, 0.0, 0.0]",17.0,47.0,"[1.9792438745498657, -2.7342734336853027, -0.3...",4.0
3,0.838661,0.680978,0.628201,0.628201,1.332256,1.332256,"[0.23759283125400543, -1.0364185571670532, -0....","[464.00299072265625, -2500.896484375, -1899.69...",130.0,2112.0,...,0.0,0.0,0.0,0,0.000000,"[0.0, 0.0, 0.0]",0.0,9.0,"[0.11302614212036133, -0.5217794179916382, -0....",2.0
4,16.057261,15.944997,15.547081,15.547081,15.783525,15.956574,"[9.091797828674316, -11.667522430419922, -5.98...","[8.460753440856934, -10.84792709350586, -5.564...",11.0,11.0,...,0.0,1.0,1.0,1,0.092946,"[0.0, 0.0, 0.0]",395.0,2.0,"[9.150219917297363, -11.740851402282715, -6.02...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2.976279,3.133054,2.605946,2.605945,2.652252,2.652252,"[1.8331600427627563, 1.1702879667282104, 1.518...","[1852.3895263671875, 1182.2166748046875, 1533....",11.0,22.0,...,3984.0,0.0,0.0,0,0.000000,"[881.5418090820312, 467.8494567871094, 694.800...",99.0,0.0,"[2.157214403152466, 1.1451619863510132, 1.7009...",0.0
11,1.780757,1.383310,1.581946,1.309141,1.610056,1.610056,"[1.2374122142791748, 0.44187161326408386, 0.93...","[2102.169921875, 762.1708374023438, 1584.96350...",-11.0,22.0,...,3984.0,0.0,0.0,0,0.000000,"[881.5418090820312, 467.8494567871094, 694.800...",60.0,0.0,"[1.292625069618225, 0.6847134828567505, 1.0155...",0.0
12,NaN,NaN,0.247770,NaN,0.252173,0.252173,"[0.13709954917430878, 0.19464954733848572, 0.0...","[1248.6241455078125, 1784.5906982421875, 760.8...",NaN,22.0,...,3984.0,0.0,0.0,0,0.000000,"[nan, nan, nan]",7.0,0.0,"[nan, nan, nan]",NaN
13,NaN,NaN,0.129988,NaN,0.132297,0.132297,"[0.034534819424152374, 0.11715523898601532, 0....","[633.2728881835938, 2148.301513671875, 932.243...",NaN,22.0,...,3984.0,0.0,0.0,0,0.000000,"[nan, nan, nan]",10.0,0.0,"[nan, nan, nan]",NaN
